In [2]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input, LSTM, RepeatVector
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Model
import keras
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [36]:
top_words = 5000
(X_train1, y_train1), (X_test1, y_test1) = imdb.load_data(num_words=top_words)

In [37]:
max_review_length = 500
X_train1 = sequence.pad_sequences(X_train1, maxlen=max_review_length)
X_test1 = sequence.pad_sequences(X_test1, maxlen=max_review_length)

In [38]:
timesteps = 5
input_dim = max_review_length
latent_dim =8
len(X_train1[0])

500

In [39]:
X_train1.shape

(25000, 500)

## Experimentation with simple seq2seq auto-encoder
Here I pass the same sentence (seq of words) as input as well as target for the training, apart for testing syntax it doesn't really have any meaning, simply following what was shown in the diagram in the assignment sheet. A more interesting version of it would be input and target pairs which are at offset of 1(or someother number), target being ahead of input. We can then hope that the model will learn to predict wht usually comes after a given set of sequences. Not the main problem.

In [40]:
X_train1 = X_train1.reshape(-1, 5, 500)
X_test1 = X_test1.reshape(-1, 5, 500)
y_train1 = X_train1
y_test1 = X_test1
print(X_train1.shape, X_test1.shape, y_train1.shape, y_test1.shape)

(5000, 5, 500) (5000, 5, 500) (5000, 5, 500) (5000, 5, 500)


In [8]:
#just encoder
inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)
encoder = Model(inputs, encoded)

In [20]:
#decoder part
decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

In [21]:
#autoencoder
sequence_autoencoder = Model(inputs, decoded)

In [22]:
sequence_autoencoder.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
sequence_autoencoder.fit(X_train1, y_train1, batch_size=200, nb_epoch=15, verbose=1)

Epoch 1/15
 400/5000 [=>............................] - ETA: 2s - loss: 0.0116 - acc: 0.0000e+00

/home/udion/Misc/anaconda3/envs/DeepCV/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


5000/5000 [==============================] - 2s 401us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 2/15
5000/5000 [==============================] - 2s 391us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 3/15
5000/5000 [==============================] - 2s 398us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 4/15
5000/5000 [==============================] - 2s 393us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 5/15
5000/5000 [==============================] - 2s 395us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 6/15
5000/5000 [==============================] - 2s 399us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 7/15
5000/5000 [==============================] - 2s 394us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 8/15
5000/5000 [==============================] - 2s 394us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 9/15
5000/5000 [==============================] - 2s 400us/step - loss: 0.0114 - acc: 1.6000e-04
Epoch 10/15
5000/5000 [==============================] - 2s 400us/step - loss: 0.011

## The second part
This part of the problem only uses the embeddings from the encoder and not anything after that, It isn't using the concept of decoder, hence it's equivalent to a model which has an encoder and a dense layer at top of that, 
as can be seen below, even with just 5 epoches it was possible to get an accuracy of 82.2% with this simple model.

The dense layer has a single node as output with softmax, which performs the classification whether the input was a good review or bad.

In [29]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [31]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [32]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [33]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [34]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [35]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('score :{}'.format(score))
print('acc :{}'.format(acc))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 151s 6ms/step - loss: 0.4640 - acc: 0.7782 - val_loss: 0.4241 - val_acc: 0.8045
Epoch 2/5
25000/25000 [==============================] - 150s 6ms/step - loss: 0.2984 - acc: 0.8778 - val_loss: 0.3834 - val_acc: 0.8311
Epoch 3/5
25000/25000 [==============================] - 152s 6ms/step - loss: 0.2165 - acc: 0.9155 - val_loss: 0.4141 - val_acc: 0.8243
Epoch 4/5
25000/25000 [==============================] - 152s 6ms/step - loss: 0.1493 - acc: 0.9454 - val_loss: 0.4741 - val_acc: 0.8290
Epoch 5/5
25000/25000 [==============================] - 35s 1ms/step
score :0.5732381032419205
acc :0.822
